In [20]:
# Run this cell only if working in Colab
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data
if os.path.exists('./sample_data'):
  !rm -r ./sample_data

repo_name = "ASL-ClassAndDetect" # Enter repo name

if not os.path.isdir('./' + repo_name):
  # Clone GitHub files to colab workspace
  git_path = 'https://github.com/christianwhollar/ASL-ClassAndDetect.git'
  !git clone "{git_path}"

  # Install dependencies from requirements.txt file
  !pip install -r requirements.txt

# Change working directory to location of notebook
if not os.path.basename(os.getcwd()) == repo_name:
  %cd "{repo_name}"

In [29]:
from google.colab import files

if 'kaggle.json' not in os.listdir():
  files.upload()
  ! mkdir ~/.kaggle
  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%run setup